In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration

device = torch.device('mps' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 5
batch_size = 20
learning_rate = 0.001

# dataset has PILImage images of range [0, 1].
# We transform them to Tensors of normalised range [-1, 1]
# Min of input image = 0 -> 0-0.5 = -0.5 gets divided by 0.5 -> -1
# Max of input image = 1 -> 1-0.5 = 0.5 gets divided by 0.5 -> 1
# so it transforms your data in a range of [-1, 1]

#CIFAR10 is a dataset of 50,000 32x32 color training images, labeled over 10 categories, and 10,000 test images.
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms,)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms,)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck') # Hard code names

def imshow(img):
  img = img / 2 + 0.5 # unnormalize
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()
  
# get some random training images
dataiter = iter(train_loader)
# images, labels = dataiter.next()
images, labels = next(dataiter)

# dataiter = iter(dataloader)
# data = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))

class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5) # 3 input channels, 6 output channels, 5x5 square convolution
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    
  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)
    x = x.view(-1, 16 * 5 * 5)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    return x
  
model = ConvNet().to(device)

loss = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader) 
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # origin shape: [4, 3, 32, 32] = 4, 3, 1024
    # input_layer: 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)
    
    # Forward pass
    outputs = model(images)
    l = loss(outputs, labels)
    
    # Backward and optimsie
    optimiser.zero_grad()
    l.backward()
    optimiser.step()

    if (i+1) % 2000 == 0:
      print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {l.item():.4f}')
      
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    # max returns (value, index)
    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()
    
    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1
      
  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc} %')
  
  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')

Files already downloaded and verified
Files already downloaded and verified


TypeError: 'module' object is not callable